# 7 deploy containers to turtlebot3

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/roboticbase-core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/roboticbase-core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/minikube/env

## set external ip addr of minikube machine

In [ ]:
export EXTERNAL_HOST_IPADDR=$(ifconfig en0 | awk '/inet / {print $2}');echo ${EXTERNAL_HOST_IPADDR}

## install libraries for `tools/deploy_yaml.py`

In [ ]:
pip install -r tools/requirements.txt

## prepare turtlebot3
You have to setup common procedure at first. And then, you have to setup either turtlebot3_simulator (A.) or actual turtlebot3 robot (B.).

### common procedure: deploy ros-master

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/ros-master:0.2.0 ros/ros-master

In [ ]:
docker push ${REPOSITORY}/tech-sketch/ros-master:0.2.0

#### apply Service

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py ${PJ_ROOT}/ros/ros-master/yaml/ros-master-service.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get services -l app=ros-master
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
ros-master   ClusterIP   None         <none>        11311/TCP   2m
```

#### apply Deployment 

In [ ]:
envsubst < ${PJ_ROOT}/ros/ros-master/yaml/ros-master-deployment-minikube.yaml > /tmp/ros-master-deployment-minikube.yaml
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py /tmp/ros-master-deployment-minikube.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}
rm /tmp/ros-master-deployment-minikube.yaml

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get deployments -l app=ros-master
NAME         DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
ros-master   1         1         1            1           20s
```

```bash
turtlebot3@turtlebot3:~$ kubectl get pods -l app=ros-master
NAME                         READY     STATUS    RESTARTS   AGE
ros-master-b479df4b5-2wq74   1/1       Running   0          22s
```

#### confirm log

```bash
turtlebot3@turtlebot3:~$ kubectl logs -f $(kubectl get pods -l app=ros-master -o template --template "{{(index .items 0).metadata.name}}")
```

### common procedure:  deploy fiware-ros-turtlebot3-bridge

#### build container image

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/fiware-ros-turtlebot3-bridge:0.2.0 ros/fiware-ros-turtlebot3-bridge

In [ ]:
docker push ${REPOSITORY}/tech-sketch/fiware-ros-turtlebot3-bridge:0.2.0

#### apply Secret

In [ ]:
export MQTT_YAML_BASE64=$(cat << __EOS__ | envsubst | base64
mqtt:
  host: "${EXTERNAL_HOST_IPADDR}"
  port: 1883
  username: "ros"
  password: "${MQTT__ros}"
  use_ca: false
__EOS__)

In [ ]:
envsubst < ${PJ_ROOT}/ros/fiware-ros-turtlebot3-bridge/yaml/fiware-ros-turtlebot3-bridge-secret.yaml > /tmp/fiware-ros-turtlebot3-bridge-secret.yaml
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py /tmp/fiware-ros-turtlebot3-bridge-secret.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}
rm /tmp/fiware-ros-turtlebot3-bridge-secret.yaml

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get secrets -l app=ros-bridge
NAME                 TYPE      DATA      AGE
ros-bridge-secrets   Opaque    1         35m
```

#### apply ConfigMap

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py ${PJ_ROOT}/ros/fiware-ros-turtlebot3-bridge/yaml/fiware-ros-turtlebot3-bridge-configmap.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get configmaps -l app=ros-bridge
NAME                    DATA      AGE
ros-bridge-configmaps   2         33m
```

#### apply Service

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py ${PJ_ROOT}/ros/fiware-ros-turtlebot3-bridge/yaml/fiware-ros-turtlebot3-bridge-service.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get services -l app=ros-bridge
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
ros-bridge   ClusterIP   None         <none>        11311/TCP   33m
```

#### apply Deployment

In [ ]:
envsubst < ${PJ_ROOT}/ros/fiware-ros-turtlebot3-bridge/yaml/fiware-ros-turtlebot3-bridge-deployment-minikube.yaml > /tmp/fiware-ros-turtlebot3-bridge-deployment-minikube.yaml
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py /tmp/fiware-ros-turtlebot3-bridge-deployment-minikube.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}
rm /tmp/fiware-ros-turtlebot3-bridge-deployment-minikube.yaml

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get deployments -l app=ros-bridge
NAME         DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
ros-bridge   1         1         1            1           31m
```

```bash
turtlebot3@turtlebot3:~$ kubectl get pods -l app=ros-bridge
NAME                          READY     STATUS    RESTARTS   AGE
ros-bridge-7fdcdc7558-j66ng   1/1       Running   0          31m
```

#### confirm log

```bash
turtlebot3@turtlebot3:~$ kubectl logs -f $(kubectl get pods -l app=ros-bridge -o template --template "{{(index .items 0).metadata.name}}")
```

### common procedure:  deploy fiware-ros-turtlebot3-operator

#### build container image

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/fiware-ros-turtlebot3-operator:0.2.0 ros/fiware-ros-turtlebot3-operator

In [ ]:
docker push ${REPOSITORY}/tech-sketch/fiware-ros-turtlebot3-operator:0.2.0

#### apply ConfigMap

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py ${PJ_ROOT}/ros/fiware-ros-turtlebot3-operator/yaml/fiware-ros-turtlebot3-operator-configmap.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get configmaps -l app=turtlebot3-operator
NAME                             DATA      AGE
turtlebot3-operator-configmaps   1         18s
```

#### apply Service

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py ${PJ_ROOT}/ros/fiware-ros-turtlebot3-operator/yaml/fiware-ros-turtlebot3-operator-service.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get services -l app=turtlebot3-operator
NAME                  TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
turtlebot3-operator   ClusterIP   None         <none>        11311/TCP   15s
```

#### apply Deployment 

In [ ]:
envsubst < ${PJ_ROOT}/ros/fiware-ros-turtlebot3-operator/yaml/fiware-ros-turtlebot3-operator-deployment-minikube-wide.yaml > /tmp/fiware-ros-turtlebot3-operator-deployment-minikube-wide.yaml
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py /tmp/fiware-ros-turtlebot3-operator-deployment-minikube-wide.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}
rm /tmp/fiware-ros-turtlebot3-operator-deployment-minikube-wide.yaml

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get deployments -l app=turtlebot3-operator
NAME                  DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
turtlebot3-operator   1         1         1            1           1m
```

```bash
turtlebot3@turtlebot3:~$ kubectl get pods -l app=turtlebot3-operator
NAME                                   READY     STATUS    RESTARTS   AGE
turtlebot3-operator-85b8bbbb88-kwstt   1/1       Running   0          32s
```

#### confirm log

```bash
turtlebot3@turtlebot3:~$ kubectl logs -f $(kubectl get pods -l app=turtlebot3-operator -o template --template "{{(index .items 0).metadata.name}}")
```

### A. turtlebot3 simulator: deploy turtlebot3

#### confirm the UID of `turtlebot3`

```bash
turtlebot3@turtlebot3:~$ echo ${UID}
1000
```

In [ ]:
export TURTLEBOT3_USER=turtlebot3
export TURTLEBOT3_UID=1000

#### build container image

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/turtlebot3-fake:0.2.0 --build-arg TURTLEBOT3_USER=${TURTLEBOT3_USER} --build-arg TURTLEBOT3_UID=${TURTLEBOT3_UID} ros/turtlebot3-fake

In [ ]:
docker push ${REPOSITORY}/tech-sketch/turtlebot3-fake:0.2.0

#### apply Service

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py ${PJ_ROOT}/ros/turtlebot3-fake/yaml/turtlebot3-fake-service.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get services -l app=turtlebot3-fake
NAME              TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
turtlebot3-fake   ClusterIP   None         <none>        11311/TCP   11s
```

#### apply Deployment 

In [ ]:
envsubst < ${PJ_ROOT}/ros/turtlebot3-fake/yaml/turtlebot3-fake-deployment-minikube.yaml > /tmp/turtlebot3-fake-deployment-minikube.yaml
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py /tmp/turtlebot3-fake-deployment-minikube.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}
rm /tmp/turtlebot3-fake-deployment-minikube.yaml

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get deployments -l app=turtlebot3-fake
NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
turtlebot3-fake   1         1         1            1           29s
```

```bash
turtlebot3@turtlebot3:~$ kubectl get pods -l app=turtlebot3-fake
NAME                              READY     STATUS    RESTARTS   AGE
turtlebot3-fake-df8bbc6f5-cpftd   1/1       Running   0          38s
```

#### confirm log

```bash
turtlebot3@turtlebot3:~$ kubectl logs -f $(kubectl get pods -l app=turtlebot3-fake -o template --template "{{(index .items 0).metadata.name}}")
```

### A. (alterntive) turtlebot3 simulator: run turtlebot3-fake directly

When the pod of `turtlebot3-fake` can't start due to the trouble of OpenGL, you should try to run `turtlebot3-fake` directry 

#### install telepresence to turtlebot3-fake machine

```bash
turtlebot3@turtlebot3-fake:~$ curl -s https://packagecloud.io/install/repositories/datawireio/telepresence/script.deb.sh | sudo bash
```

```bash
turtlebot3@turtlebot3-fake:~$ sudo apt install --no-install-recommends telepresence
```

#### forward `ros-master` port

```bash
turtlebot3@turtlebot3-fake:~$ kubectl port-forward $(kubectl get pods -l app=ros-master -o template --template "{{(index .items 0).metadata.name}}") 11311:11311
```

#### start `turtlebot3-fake` on telepresence shell

* cd `ros_ws`

```bash
turtlebot3@turtlebot3-fake:~$ telepresence --run-shell
```

```bash
@minikube|bash-4.3$ cd <<path/to/ros_workspace>>
@minikube|bash-4.3$ source /opt/ros/kinetic/setup.bash 
@minikube|bash-4.3$ source ./devel/setup.bash 
@minikube|bash-4.3$ export ROS_MASTER_URI=http://localhost:11311
@minikube|bash-4.3$ export ROS_HOSTNAME=172.17.0.1
@minikube|bash-4.3$ catkin_make
@minikube|bash-4.3$ roslaunch turtlebot3_fake turtlebot3_fake.launch 
```

### B. actual turtlebot3 robot: deploy turtlebot3

#### set ROS workspace of turtlebot3

In [ ]:
export TURTLEBOT3_WORKSPACE=/home/turtlebot3/catkin_ws

#### build container image

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/turtlebot3-bringup:0.2.0 ros/turtlebot3-bringup

In [ ]:
docker push ${REPOSITORY}/tech-sketch/turtlebot3-bringup:0.2.0

#### apply Service

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py ${PJ_ROOT}/ros/turtlebot3-bringup/yaml/turtlebot3-bringup-service.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get services -l app=turtlebot3-bringup
NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
turtlebot3-bringup   ClusterIP   None         <none>        11311/TCP   13s
```

#### apply Deployment 

In [ ]:
envsubst < ${PJ_ROOT}/ros/turtlebot3-bringup/yaml/turtlebot3-bringup-deployment-minikube.yaml > /tmp/turtlebot3-bringup-deployment-minikube.yaml
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
./tools/deploy_yaml.py /tmp/turtlebot3-bringup-deployment-minikube.yaml http://${HOST_IPADDR}:8080 ${TOKEN} ${FIWARE_SERVICE} ${DEPLOYER_SERVICEPATH} ${DEPLOYER_TYPE} ${DEPLOYER_ID}
rm /tmp/turtlebot3-bringup-deployment-minikube.yaml

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get deployments -l app=turtlebot3-bringup
NAME                 DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
turtlebot3-bringup   1         1         1            1           1m
```

```bash
turtlebot3@turtlebot3:~$ kubectl get pods -l app=turtlebot3-bringup
NAME                                  READY     STATUS    RESTARTS   AGE
turtlebot3-bringup-5c7b59c9b4-c56kj   1/1       Running   0          1m
```

#### confirm log

```bash
turtlebot3@turtlebot3:~$ kubectl logs -f $(kubectl get pods -l app=turtlebot3-bringup -o template --template "{{(index .items 0).metadata.name}}")
```